This notebook will illustrate data cleaning techniques for the Austin Texas Resturaunt Inspections Dataset https://data.world/adamhelsinger/austin-restaurant-inspections

In [13]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('/home/novascott/Dev/NYC-OpenData/Restaurant_Inspection_Scores.csv')
df.columns = list(map(lambda x: x.replace(' ', '_'), df.columns))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25033 entries, 0 to 25032
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Restaurant_Name      25033 non-null  object
 1   Zip_Code             25033 non-null  object
 2   Inspection_Date      25033 non-null  object
 3   Score                25033 non-null  int64 
 4   Address              25033 non-null  object
 5   Facility_ID          25033 non-null  int64 
 6   Process_Description  25033 non-null  object
dtypes: int64(2), object(5)
memory usage: 1.3+ MB


In [3]:
#Check for and Remove Duplicate Rows
df.duplicated().sum()

1

In [4]:
df = df.drop_duplicates()

In [6]:
# Restructing Dataframe:
applied_df = df['Address']
# Splitting Address Column to 3 seperate Columns
applied_df = applied_df.str.split(pat='\n', expand=True)
applied_df.columns = ['St Address','City','Location']
applied_df = applied_df['Location'].apply(lambda x: x.replace('(','').replace(')',''))
# Split Location to serperate longitude and latitude columns
ltlng_df = applied_df.str.split(pat=',', expand=True)
ltlng_df.columns = ['Latitude','Longitude']
# Inconsistant data, changing strings with "na", "nan", etc with numpy nan values
ltlng_df.replace(to_replace=['','nan','NaN',None], value=np.nan, inplace=True)
#  Properly casting types to specified columns
ltlng_df = ltlng_df.astype(float)
df = pd.concat([df, applied_df, ltlng_df], axis='columns')

In [7]:
# Castinf column as datetime format
df['Inspection_Date'] = pd.to_datetime(df['Inspection_Date'],format='%m/%d/%Y')
# Creating New columns year, month, day, and day of the week based datetime
df['Year'] = df['Inspection_Date'].dt.year
df['Month'] = df['Inspection_Date'].dt.month
df['Day'] = df['Inspection_Date'].dt.day
df['Weekday'] = df['Inspection_Date'].dt.day_name()

In [8]:
# Creating Boolean Type Column if a resturaunt is a franchise.
#Looks at the resturaunt name column and looks for a "#" string that suggests franchising
franchise_df = df['Restaurant_Name']
franchise_df = franchise_df.str.contains('#', regex=False)
franchise_df.name = 'Franchise'
df = pd.concat([df, franchise_df], axis='columns')
df.set_index('Restaurant_Name')
df.head()

,Restaurant_Name,Zip_Code,Inspection_Date,Score,Address,Facility_ID,Process_Description,Location,Latitude,Longitude,Year,Month,Day,Weekday,Franchise
0,Williams Sonoma #706,78746,2015-06-16,85,"2901 S CAPITAL OF TEXAS HWY NB\nAUSTIN, TX 787...",2801932,Routine Inspection,"30.254658, -97.809462",30.254658,-97.809462,2015,6,16,Tuesday,True
1,Whisler's,78702,2015-01-06,83,"1816 E 6TH ST\nAUSTIN, TX 78702\n(30.261738, -...",10928995,Routine Inspection,"30.261738, -97.722685",30.261738,-97.722685,2015,1,6,Tuesday,False
2,Whataburger #347,78757,2016-06-03,97,"2801 W ANDERSON LN\nAUSTIN, TX 78757\n(30.3589...",2800287,Routine Inspection,"30.358957, -97.736956",30.358957,-97.736956,2016,6,3,Friday,True
3,Westin Austin Downtown,78701,2016-03-07,76,"310 E 5TH ST\nAUSTIN, TX 78701\n(30.26632, -97...",11377565,Routine Inspection,"30.26632, -97.740282",30.266320,-97.740282,2016,3,7,Monday,False
4,Whistle Stop Grocery/La Mexicana,78753,2015-05-18,81,"834 E RUNDBERG LN Bldg B\nAUSTIN, TX 78753\n(3...",2803636,Routine Inspection,"30.35723, -97.686114",30.357230,-97.686114,2015,5,18,Monday,False


In [9]:
# Checking for consistant data in the zip code column 
df.Zip_Code.unique()

array(['78746', '78702', '78757', '78701', '78753', '78751', '78723',
       '78752', '78733', '78703', '78749', '78617', '78745', '78759',
       '78758', '78729', '78705', '78704', '78728', '78731', '78741',
       '78660', '78613', '78750', '78748', '78744', '78721', '78724',
       '78652', '78717', '78669', '78726', '78732', '78727', '78722',
       '78734', '78754', '78756', '78719', '78653', '78735', '78738',
       '78739', '78730', '78737', '78612', '78747', '78725', '78736',
       '78654', '78620', '78742', '6\nAustin, TX  78741', '78641',
       'Hwy 290 E\nAustin TX  78723', '78615'], dtype=object)

In [14]:
# Taking zip code columns strings and looking for zip codes using regex 
df['Zip_Code'] = df['Zip_Code'].apply(lambda x: "".join(map(str,re.findall("[0-9]{5}(?:-[0-9]{4})?", x))))

In [15]:
# Double Checking for consistant data
df.Zip_Code.unique()

array(['78746', '78702', '78757', '78701', '78753', '78751', '78723',
       '78752', '78733', '78703', '78749', '78617', '78745', '78759',
       '78758', '78729', '78705', '78704', '78728', '78731', '78741',
       '78660', '78613', '78750', '78748', '78744', '78721', '78724',
       '78652', '78717', '78669', '78726', '78732', '78727', '78722',
       '78734', '78754', '78756', '78719', '78653', '78735', '78738',
       '78739', '78730', '78737', '78612', '78747', '78725', '78736',
       '78654', '78620', '78742', '78641', '78615'], dtype=object)

In [16]:
# Saving externally as XLSX File
df.to_excel('.../Restaurant_Inspection_Scores(Clean).xlsx',header=df.columns,index=False,index_label=df.columns)